# Gerador e Verificador de Assinaturas Virtuais

Geradores de assinaturas digitais são ferramentas as quais garantem a autenticidade e/ou a integridade de uma dada mensagem. Para isso, utiliza-se técnicas de criptografia, normalmente criptografia assimétrica, onde uma chave privada é empregada para assinar digitalmente um dado, e o hash da mensagem é incorporado na assinatura como uma forma de validação do conteúdo.

Já verificadores de assinaturas são responsáveis por confirmar a autenticidade e/ou a integridade de uma assinatura digital. Considerando um cenário de criptografia assimétrica como o dado acima, o verificador utiliza a chave pública correspondente do emissor para decifrar a mensagem cifrada, caso a mensagem realmente seja do emissor declaro, o hash deste texto em claro deve ser correspondente ao hash enviado pelo o emissor.

Assim, neste notebook iremos implementar um gerador e verificador de assinatura digitais de documentos, utilizando técnicas avançadas em segurança da computação. Para isso, o projeto consta com quatro módulos principais: geração de chaves,  encriptação e decriptação, assinatura e verificação.

O primeiro módulo será responsável pela geração de chaves criptográficas com base no algoritmo RSA, onde essas chaves serão derivadas de números primos de 1024 bits. O segundo módulo dedica-se ao processo de encriptação e decriptação de mensagens utilizando o OAEP (Optimal Asymmetric Encryption Padding). O terceiro realiza o cálculo do hash da mensagem e formatação do resultado em BASE64. Por fim, o quarto módulo sumariza a verificação de assinaturas digitais com um exemplo prático.


## Geração de Chaves

Primeiramente começaremos nosso projeto com a parte mais crucial nos sistemas modernos de criptografia, a geração das chaves. As chaves são responsáveis por garantir a confidencialidade e a integridade de qualquer algoritmo de criptografia moderno, então é necessário uma alta atenção para se evitar qualquer tipo de padrão ou rastreabilidade em sua geração. Para isso, usaremos duas bibliotecas que irão nos trazer a aleatoriedade necessária para a geração das Chaves.

In [184]:
from random import getrandbits, randrange

O algoritmo que será responsável pela encriptação e decriptação do nosso projeto será o algoritmo de criptografia assimétrica RSA, amplamente reconhecido por sua segurança e aplicabilidade em sistemas modernos de criptografia assimétrica. 

Dito isso, por ser assimétrico, iremos gerar duas chaves, uma pública e uma privada. A chave pública será composta por n=p×q,  onde p e q são números primos com no mínimo 1024 bits e por "e", o qual será um número inteiro escolhido que satisfaça a propriedade 1< "e" < ϕ(n) e sendo coprimo de ϕ(n) = (p−1)×(q−1). Já a chave privada será representada por "d", calculado como "d"≡ "e"⁻¹ (mod  ϕ(n)), sendo "d" o inverso modular de "e" em relação a ϕ(n).

Com isto, o primeiro passo será gerar os primos q e p. Faremos isso usando o teste de primalidade de Miller–Rabin, o que nos retornará dois primos diferentes entre si com uma alta probabilidade.

In [185]:
def is_prime(n: int, k=128) -> bool:
    """Teste de primalidade usando o algoritmo de Miller-Rabin."""
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2

    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def get_prime():
    """Gera um número primo de 1024 bits."""
    while True:
        p = getrandbits(1024)
        p |= 1  # Garante que o número é ímpar
        p |= 1 << 1023  # Garante o tamanho correto (1024 bits)
        if is_prime(p):
            return p
        
p = get_prime()
q = get_prime()

while p == q:
    q = get_prime()

Com p e q gerados, iremos definir algumas funções auxiliares para calcularmos a operação mdc e os valores de n e phi.

In [186]:
def gcd(a, b):
    """Calcula o máximo divisor comum (GCD) usando o algoritmo de Euclides."""
    while b:
        a, b = b, a % b
    return a

def get_n(p, q):
    """Calcula o valor de n = p * q."""
    return p * q

def get_phi(p, q):
    """Calcula o valor de phi = (p - 1) * (q - 1)."""
    return (p - 1) * (q - 1)

n = get_n(p, q)
phi = get_phi(p, q)

Agora, iremos definir duas função que irão encapsular as equações de geração de "e" e "d"

In [187]:
def choose_e(phi):
    """Escolhe um valor de 'e' que seja coprimo a 'phi'."""
    e = 2
    while e < phi and gcd(e, phi) != 1:
        e += 1
    return e

def get_d(e, phi):
    """Encotra o inverso modular de 'e'."""
    return pow(e, -1, phi)

e = choose_e(phi)
d = get_d(e, phi)

Pronto, com "n", "e" e "d" estamos pronto para gerarmos nossas chaves com a função generate_keys();

In [188]:
def generate_keys(n, e, d):
    """Gera as chaves pública e privada."""
    public_key = (e, n)
    private_key = (d, n)

    return public_key, private_key

pk, sk = generate_keys(n, e, d)

print(f"Chave Pública: {pk}. \n Chave privada: {sk}")

Chave Pública: (3, 13935534021441327332743015816144125374915383965472508066227515300564428844698338731558061881012613436368537336727952023568933429884111269827265553661077104135013224768688147474475973404731393710285316000120084323892464535567391040785231402075647286267208315606288702660460346937606401824350265839082830715230853227880709517674177315026222160008170064650950370669656975078741690358639967822017971172242136862977835683243280740503074546462911030951914429618211465016875697930815662226091883105702457427286747301088730713822315458403327435906878311584298711566317680245006600242986869957918772259115220142569265165565431). 
 Chave privada: (9290356014294218221828677210762750249943589310315005377485010200376285896465559154372041254008408957579024891151968015712622286589407513218177035774051402756675483179125431649650648936487595806856877333413389549261643023711594027190154268050431524178138877070859135106973564625070934549566843892721887143487078036824166418798415436318359028915

## Encriptação e Decriptação

### RSA

In [189]:
class rsa:
    def __init__(self):
        # Gera as chaves pública e privada
        self.public_key, self.private_key = pk, sk

    def encrypt(self, message: int):
        """Criptografa uma mensagem usando a chave privada."""
        d, n = self.private_key
        result = pow(message, d, n)  # (m^d) mod n
        return str(result)

    def decrypt(self, message: int):
        """Descriptografa uma mensagem usando a chave pública."""
        e, n = self.public_key
        result = pow(message, e, n)  # (c^e) mod n
        return str(result)


In [190]:
class oaep:
  #usando uma seguranca de 10245 bits
  #podemos usar k0 como o mesmo tamanho da funcao de hash que usaremos em m+k1
  #Como sera usada a sha-512 colocamos k0 como 512
  k0 = 512

  #Dessa forma k1 deve ser 256 para que m + k1 + k0 = 1024
  k1 = 256

  x = None
  y = None

  def encrypt(self, message: int):
    #padding de k1 0s na mensagem
    message = message << self.k1
    #geracao de k0 bits aleatorios
    r = str(getrandbits(self.k0))

    #utilizamos o sha3-512 e fazemos um XOR entre os resultados. para obter x
    self.x = message ^ int(sha3.hash_512(r),16)

    #como X tem o mesmo tamanho de r podemos usar novamente o sha3-512 para fazer o segundo hash e obter y
    self.y = int(r) ^ int(sha3.hash_512(str(self.x)),16)

    #apos isso concatenamos os dois valores
    return str(self.x) + str(self.y)



  def decrypt(self, message: str):
    #separamos os valores x e y
    #sabemos que ambos os lados do input tem 154 digitos logo essa tarefa fica facil
    x = message[:154]
    y = message[154:]

    #primeiro recuperamos r
    r = int(y) ^ int(sha3.hash_512(x),16)
    #Com r podemos recuperar a mensagem com o padding de 0s
    v = int(x) ^ int(sha3.hash_512(str(r)),16)
    #logo apenas resta retirar esses 0s da mensagem final
    return v >> self.k1



## Assinatura

### Hash

In [191]:
from hashlib import sha3_256, sha3_512
class sha3:
  #funcao de hash de 256 bits
  @staticmethod
  def hash_256(message):
    if type(message) == int:
      message = str(message)
    return sha3_256(message.encode()).hexdigest()
  # funcao de hash de 512 bits
  @staticmethod
  def hash_512(message: str):
    if type(message) == int:
      message = str(message)
    return sha3_512(message.encode()).hexdigest()


In [192]:
h = sha3.hash_512("123")
print(h)
h2 = sha3.hash_512(str(123))
print(h2)

48c8947f69c054a5caa934674ce8881d02bb18fb59d5a63eeaddff735b0e9801e87294783281ae49fc8287a0fd86779b27d7972d3e84f0fa0d826d7cb67dfefc
48c8947f69c054a5caa934674ce8881d02bb18fb59d5a63eeaddff735b0e9801e87294783281ae49fc8287a0fd86779b27d7972d3e84f0fa0d826d7cb67dfefc


In [193]:
o = oaep()
message = o.encrypt(1234)
print(message)
print(o.decrypt(message))

91227161193770381059105706066422985949365231602782318340431108282146993439847135182695579030552868695198872981859364502369091538208447318774925849347755088374350005711592843151130204653469129393697914158153302467736672526781087090982993343486223408404517538010973378172586723283629046278280339372134786725543
1234


### BASE64

In [194]:
class base_convert:

  #converte para base 64
  @staticmethod
  def format(message: int):
    #Usa uma string de referencia
    base64_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    res = ""
    #Enquanto a mensagem nao se tornar 0 realiza o metodo de divisoes sucessivas
    while message > 0:
      remainder = message % 64
      res = base64_chars[remainder] + res
      message = message // 64
    return res


  @staticmethod
  def parse(base64_message: str):
    #Tambem usa uma string de referebncia
    base64_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    res = 0
    # procura o caracter encontrado na string e coloca e soma com o resultado multiplicado por 64
    # essa multiplicacao por 64 fara com que os termos sejam multiplicados por 64 o numero correto de vezes
    for char in base64_message:
      res = res * 64 + base64_chars.index(char)
    return res



In [195]:
v = base_convert.format(1234)
print(v)
print(base_convert.parse(v))

TS
1234


## Verificação de Assinatura

In [196]:
# Instanciação da classe RSA
cy = rsa()

plain_text = 1234543

# Calculo do hash 256 da mensagem
h = int(sha3.hash_256(str(plain_text)), 16)

# Encryptação desse hash
e = cy.encrypt(h)

# Concatenação entre plain_text e o hash encriptado
message_to_encode = str(plain_text) + str(e)

# Conversão para a Base64
encoded_message = base_convert.format(int(message_to_encode))

print("Mensagem codificada em base64:", encoded_message)

# Parse da base64
decoded_message = str(base_convert.parse(encoded_message))

# Separação entre o plain_text e o hash encriptado
decoded_plain_text = decoded_message[:len(str(plain_text))]

encrypted_hash = decoded_message[len(str(plain_text)):]

# Decryptação do hash encriptado
decrypted_hash = cy.decrypt(int(encrypted_hash))

# Comparação entre os hashs para verificação da assinatura
print("Hash original:", h)
print("Hash decodificado:", decrypted_hash)
print("Verificação de assinatura:", int(decrypted_hash) == h)

Mensagem codificada em base64: F1DqAMDDZP0gmOXMJ1tTzmi07lPYcnSUTB7GMirffG2O2TBzQQ5eXPxvJFlWdC2T0xQ/hrSRh/J8i079TyNwRKwZPNHmv9E940N6A7OjrvU3ykwueK6i1D22LQ+Xz4XJGYBFPYKn1Lj/agGiuaGKfSJx20uCeUDs3WG7/HKrxFIIQSsTXkK4MsiENYoR0et8t73b5kEbVXiONe3cZN7z6c0JB4F/06vffDERM5+BgQNOy5D1es2pClRk+OrJBkGizW2s2qMuaktwwbebQZik2cm9Opy8Ss7KQtlCxK84TxRvvTOn2jEIrdbYG96Pt3siZvciTOcDiHObuA8OFEKzJAKf7
Hash original: 100605913033220575484136493710894157805945673824467054724527238753657734088666
Hash decodificado: 100605913033220575484136493710894157805945673824467054724527238753657734088666
Verificação de assinatura: True
